### rock_samplesという名前でデータをインポートし、最初の5つを出す、またinfoでデータの全容をつかんでください。

In [1]:
import pandas as pd

In [2]:
rock_samples=pd.read_csv('rocksamples.csv')

In [3]:
rock_samples.head()

,ID,Mission,Type,Subtype,Weight (g),Pristine (%)
0,10001,Apollo11,Soil,Unsieved,125.8,88.36
1,10002,Apollo11,Soil,Unsieved,5629.0,93.73
2,10003,Apollo11,Basalt,Ilmenite,213.0,65.56
3,10004,Apollo11,Core,Unsieved,44.8,71.76
4,10005,Apollo11,Core,Unsieved,53.4,40.31


In [4]:
rock_samples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2229 entries, 0 to 2228
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            2229 non-null   int64  
 1   Mission       2229 non-null   object 
 2   Type          2229 non-null   object 
 3   Subtype       2226 non-null   object 
 4   Weight (g)    2229 non-null   float64
 5   Pristine (%)  2229 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 104.6+ KB


### weightをg単位からkg単位に変換してください。また、表記もkg単位に表記し直してください

In [5]:
rock_samples['Weight (g)']=rock_samples['Weight (g)'].map((lambda weight:weight*0.001))

In [6]:
rock_samples=rock_samples.rename(columns={'Weight (g)':'Weight (kg)'})

In [7]:
rock_samples

,ID,Mission,Type,Subtype,Weight (kg),Pristine (%)
0,10001,Apollo11,Soil,Unsieved,0.12580,88.36
1,10002,Apollo11,Soil,Unsieved,5.62900,93.73
2,10003,Apollo11,Basalt,Ilmenite,0.21300,65.56
3,10004,Apollo11,Core,Unsieved,0.04480,71.76
4,10005,Apollo11,Core,Unsieved,0.05340,40.31
...,...,...,...,...,...,...
2224,79528,Apollo17,Breccia,Regolith,0.00238,100.00
2225,79529,Apollo17,Breccia,Regolith,0.00184,100.00
2226,79535,Apollo17,Breccia,Regolith,0.00169,100.00
2227,79536,Apollo17,Breccia,Regolith,0.00166,100.00


### 新しくmissionというデータフレームを作成し、missionカラムを作成してください。

In [8]:
missions=pd.DataFrame()
missions['Mission']=rock_samples['Mission'].unique()
missions

,Mission
0,Apollo11
1,Apollo12
2,Apollo14
3,Apollo15
4,Apollo16
5,Apollo17


### missionデータフレームにおいて、各ミッションごとに持ち帰ったweightの合計を算出し、新たにsample weightカラムを作って代入してください

In [9]:
missions=pd.merge(missions,rock_samples.groupby('Mission')['Weight (kg)'].sum(),on='Mission')
missions=missions.rename(columns={'Weight (kg)':'sample weight(kg)'})

### 一個前のミッションとのsample weight の差を求めたweight diff カラムを作成する

In [10]:
missions['weight diff']=missions['sample weight(kg)'].diff()
missions

,Mission,sample weight(kg),weight diff
0,Apollo11,21.55424,NaN
1,Apollo12,34.34238,12.78814
2,Apollo14,41.83363,7.49125
3,Apollo15,75.39910,33.56547
4,Apollo16,92.46262,17.06352
5,Apollo17,109.44402,16.98140


In [11]:
missions=missions.fillna(0)
missions

,Mission,sample weight(kg),weight diff
0,Apollo11,21.55424,0.00000
1,Apollo12,34.34238,12.78814
2,Apollo14,41.83363,7.49125
3,Apollo15,75.39910,33.56547
4,Apollo16,92.46262,17.06352
5,Apollo17,109.44402,16.98140


### 月モジュールとコマンドモジュールの重量を列に追加して、それらのモジュールごとの差も計算し新たに列に追加しましょう。なお欠損値の部分は0で穴埋めしましょう

In [12]:
missions['Lunar module (LM)'] = {'Eagle (LM-5)', 'Intrepid (LM-6)', 'Antares (LM-8)', 'Falcon (LM-10)', 'Orion (LM-11)', 'Challenger (LM-12)'}
missions['LM mass (kg)'] = {15103, 15235, 15264, 16430, 16445, 16456}
missions['Command module (CM)'] = {'Columbia (CSM-107)', 'Yankee Clipper (CM-108)', 'Kitty Hawk (CM-110)', 'Endeavor (CM-112)', 'Casper (CM-113)', 'America (CM-114)'}
missions['CM mass (kg)'] = {5560, 5609, 5758, 5875, 5840, 5960}

In [13]:
missions['LM mass diff']=missions['LM mass (kg)'].diff()
missions['CM mass diff']=missions['CM mass (kg)'].diff()
missions=missions.fillna(0)
missions

,Mission,sample weight(kg),weight diff,Lunar module (LM),LM mass (kg),Command module (CM),CM mass (kg),LM mass diff,CM mass diff
0,Apollo11,21.55424,0.00000,Challenger (LM-12),15264,Columbia (CSM-107),5960,0.0,0.0
1,Apollo12,34.34238,12.78814,Antares (LM-8),15235,America (CM-114),5609,-29.0,-351.0
2,Apollo14,41.83363,7.49125,Falcon (LM-10),16456,Endeavor (CM-112),5840,1221.0,231.0
3,Apollo15,75.39910,33.56547,Intrepid (LM-6),16430,Yankee Clipper (CM-108),5875,-26.0,35.0
4,Apollo16,92.46262,17.06352,Orion (LM-11),16445,Kitty Hawk (CM-110),5560,15.0,-315.0
5,Apollo17,109.44402,16.98140,Eagle (LM-5),15103,Casper (CM-113),5758,-1342.0,198.0


### 各モジュールで合計した重量を示した列をつくる、欠損値は0で代入する

In [14]:
missions['Total weight (kg)']=missions['LM mass (kg)']+missions['CM mass (kg)'] 
missions

,Mission,sample weight(kg),weight diff,Lunar module (LM),LM mass (kg),Command module (CM),CM mass (kg),LM mass diff,CM mass diff,Total weight (kg)
0,Apollo11,21.55424,0.00000,Challenger (LM-12),15264,Columbia (CSM-107),5960,0.0,0.0,21224
1,Apollo12,34.34238,12.78814,Antares (LM-8),15235,America (CM-114),5609,-29.0,-351.0,20844
2,Apollo14,41.83363,7.49125,Falcon (LM-10),16456,Endeavor (CM-112),5840,1221.0,231.0,22296
3,Apollo15,75.39910,33.56547,Intrepid (LM-6),16430,Yankee Clipper (CM-108),5875,-26.0,35.0,22305
4,Apollo16,92.46262,17.06352,Orion (LM-11),16445,Kitty Hawk (CM-110),5560,15.0,-315.0,22005
5,Apollo17,109.44402,16.98140,Eagle (LM-5),15103,Casper (CM-113),5758,-1342.0,198.0,20861


### ペイロードに占めるクルー(生産者、人間や採取物)にかけられる重量の割合、クルーに占める採取物の割合、ペイロードに占める採取物の占める割合を計算してそれぞれ列にしてください

In [15]:
saturnVPayload = 43500
missions['Crewed area : Payload']=missions['Total weight (kg)']/saturnVPayload
missions['Sample weight: Crewed area']=missions['sample weight(kg)']/missions['Total weight (kg)']
missions['Sample weight: Payload']=missions['sample weight(kg)']/saturnVPayload
missions

,Mission,sample weight(kg),weight diff,Lunar module (LM),LM mass (kg),Command module (CM),CM mass (kg),LM mass diff,CM mass diff,Total weight (kg),Crewed area : Payload,Sample weight: Crewed area,Sample weight: Payload
0,Apollo11,21.55424,0.00000,Challenger (LM-12),15264,Columbia (CSM-107),5960,0.0,0.0,21224,0.487908,0.001016,0.000495
1,Apollo12,34.34238,12.78814,Antares (LM-8),15235,America (CM-114),5609,-29.0,-351.0,20844,0.479172,0.001648,0.000789
2,Apollo14,41.83363,7.49125,Falcon (LM-10),16456,Endeavor (CM-112),5840,1221.0,231.0,22296,0.512552,0.001876,0.000962
3,Apollo15,75.39910,33.56547,Intrepid (LM-6),16430,Yankee Clipper (CM-108),5875,-26.0,35.0,22305,0.512759,0.003380,0.001733
4,Apollo16,92.46262,17.06352,Orion (LM-11),16445,Kitty Hawk (CM-110),5560,15.0,-315.0,22005,0.505862,0.004202,0.002126
5,Apollo17,109.44402,16.98140,Eagle (LM-5),15103,Casper (CM-113),5758,-1342.0,198.0,20861,0.479563,0.005246,0.002516


### 各ミッションごとの上記3つの割合の平均を求めてください

In [16]:
crewed_area_payload_ratio_avg=missions.groupby('Mission').mean()['Crewed area : Payload'].mean()
crewed_area_payload_ratio_avg
sample_weight_crewed_area_ratio_avg=missions.groupby('Mission').mean()['Sample weight: Crewed area'].mean()
sample_weight_payload_ratio_avg=missions.groupby('Mission').mean()['Sample weight: Payload'].mean()

### アルテミスミッション用にデータフレームを作成する

In [17]:
artemis_crewedArea = 26520
artemis_mission = pd.DataFrame({'Mission':['artemis1','artemis1b','artemis2'],
                                 'Total weight (kg)':[artemis_crewedArea,artemis_crewedArea,artemis_crewedArea],
                                 'Payload (kg)':[26988, 37965, 42955]})
artemis_mission

,Mission,Total weight (kg),Payload (kg)
0,artemis1,26520,26988
1,artemis1b,26520,37965
2,artemis2,26520,42955


### アポロ計画のデータ分析からわかった各比率をもとにアルテミスミッションにおいてどれくらいの量が必要かを算出する

In [18]:
artemis_mission['Sample_weight_from_Total_weight']=artemis_mission['Total weight (kg)']*sample_weight_crewed_area_ratio_avg
artemis_mission['Sample_weight_from_payload']=artemis_mission['Payload (kg)']*sample_weight_payload_ratio_avg
artemis_mission

,Mission,Total weight (kg),Payload (kg),Sample_weight_from_Total_weight,Sample_weight_from_payload
0,artemis1,26520,26988,76.766734,38.779584
1,artemis1b,26520,37965,76.766734,54.552649
2,artemis2,26520,42955,76.766734,61.722877


### 2つのカラムの平均を算出し、持って帰ってこれるsampleの重量の推定値を求めてください

In [19]:
artemis_mission['Estemated sample weight(kg)']=(artemis_mission['Sample_weight_from_Total_weight']+artemis_mission['Sample_weight_from_payload'])/2

In [20]:
artemis_mission

,Mission,Total weight (kg),Payload (kg),Sample_weight_from_Total_weight,Sample_weight_from_payload,Estemated sample weight(kg)
0,artemis1,26520,26988,76.766734,38.779584,57.773159
1,artemis1b,26520,37965,76.766734,54.552649,65.659691
2,artemis2,26520,42955,76.766734,61.722877,69.244806


### 石のサンプル全体に占める手つかずの石の量を計算しremaining カラムに代入してください

In [21]:
rock_samples.head()

,ID,Mission,Type,Subtype,Weight (kg),Pristine (%)
0,10001,Apollo11,Soil,Unsieved,0.1258,88.36
1,10002,Apollo11,Soil,Unsieved,5.6290,93.73
2,10003,Apollo11,Basalt,Ilmenite,0.2130,65.56
3,10004,Apollo11,Core,Unsieved,0.0448,71.76
4,10005,Apollo11,Core,Unsieved,0.0534,40.31


In [22]:
rock_samples['remaining']=rock_samples['Weight (kg)']*(rock_samples['Pristine (%)']*0.01)

In [23]:
rock_samples.describe()

,ID,Weight (kg),Pristine (%),remaining
count,2229.000000,2229.000000,2229.000000,2229.000000
mean,52058.432032,0.168253,84.512764,0.138103
std,26207.651471,0.637286,22.057299,0.525954
min,10001.000000,0.000000,0.000000,0.000000
25%,15437.000000,0.003000,80.010000,0.002432
50%,65527.000000,0.010200,92.300000,0.008530
75%,72142.000000,0.093490,98.140000,0.078240
max,79537.000000,11.729000,180.000000,11.169527


### 先程絞り込んだ手つかずの石うち希少であろう石を抽出するために平均よりもweightが重いもの、かつ半分以上手がつけられている石を絞り込んでください。それをlow_sampleというデータフレームに格納してください

In [24]:
low_sample=rock_samples.loc[(rock_samples['Weight (kg)']>=rock_samples['Weight (kg)'].mean()) & (rock_samples['Pristine (%)']<=50)]

### 石全体のデータに含まれる石の種類と、絞り込んだデータに含まれる石の種類の数を比較してください。

In [25]:
rock_samples['Type'].value_counts()

Breccia    959
Soil       813
Basalt     351
Core        56
Crustal     46
Special      4
Name: Type, dtype: int64

In [26]:
low_sample['Type'].value_counts()

Basalt     14
Breccia     8
Soil        4
Core        1
Name: Type, dtype: int64

### 絞り込んだ重要度の高いと思われるデータから種類数が特に多い2つの石を取り出し、needed_samplesとして新たにデータフレーム化してください(もう一度)

In [27]:
needed_samples=low_sample[low_sample['Type'].isin(['Basalt','Breccia'])]

In [28]:
needed_samples

,ID,Mission,Type,Subtype,Weight (kg),Pristine (%),remaining
11,10017,Apollo11,Basalt,Ilmenite,0.9730,43.71,0.425298
14,10020,Apollo11,Basalt,Ilmenite,0.4250,27.88,0.118490
15,10021,Apollo11,Breccia,Regolith,0.2500,30.21,0.075525
29,10045,Apollo11,Basalt,Olivine,0.1850,12.13,0.022441
37,10057,Apollo11,Basalt,Ilmenite,0.9190,35.15,0.323028
39,10059,Apollo11,Breccia,Regolith,0.1880,36.94,0.069447
52,10072,Apollo11,Basalt,Ilmenite,0.4470,15.22,0.068033
68,12002,Apollo12,Basalt,Olivine,1.5300,49.04,0.750312
72,12006,Apollo12,Basalt,Olivine,0.2064,0.53,0.001094
113,12047,Apollo12,Basalt,Ilmenite,0.1930,33.70,0.065041


### データフレーム化したデータの石の種類ごとの重さの合計を計算してください、同様の計算を全体の石のデータにおいても行ってください

In [29]:
needed_samples.groupby('Type').sum()

,ID,Weight (kg),Pristine (%),remaining
Type,,,,
Basalt,292125,17.4234,457.03,7.371357
Breccia,252304,10.1185,288.02,4.260812


In [30]:
rock_samples.groupby('Type').sum()

,ID,Weight (kg),Pristine (%),remaining
Type,,,,
Basalt,14111750,93.14077,27348.13,70.632524
Breccia,50325667,168.88075,84624.85,143.219561
Core,2540025,19.93587,3959.80,15.385910
Crustal,2964635,4.74469,3616.80,4.175771
Soil,45931124,87.58981,68638.38,73.812392
Special,165044,0.74410,190.99,0.606289


### crusutalの量が少ないことがわかるのでこれを重要度が高いと位置づけ、needed_samplesに追加してください。

In [31]:
needed_samples=needed_samples.append(rock_samples[rock_samples['Type'].isin(['Crustal'])])

In [32]:
needed_samples

,ID,Mission,Type,Subtype,Weight (kg),Pristine (%),remaining
11,10017,Apollo11,Basalt,Ilmenite,0.97300,43.71,0.425298
14,10020,Apollo11,Basalt,Ilmenite,0.42500,27.88,0.118490
15,10021,Apollo11,Breccia,Regolith,0.25000,30.21,0.075525
29,10045,Apollo11,Basalt,Olivine,0.18500,12.13,0.022441
37,10057,Apollo11,Basalt,Ilmenite,0.91900,35.15,0.323028
...,...,...,...,...,...,...,...
2089,78238,Apollo17,Crustal,Norite,0.00000,86.03,0.000000
2092,78256,Apollo17,Crustal,Pristine,0.00000,0.00,0.000000
2126,78517,Apollo17,Crustal,Cataclasite,0.00182,82.97,0.001510
2130,78527,Apollo17,Crustal,Cataclasite,0.00516,82.05,0.004234


In [33]:
rock_samples[rock_samples['Type'].isin(['Crustal'])]

,ID,Mission,Type,Subtype,Weight (kg),Pristine (%),remaining
497,15361,Apollo15,Crustal,Cataclastic,0.00090,66.56,0.000599
498,15362,Apollo15,Crustal,Cataclastic,0.00420,56.88,0.002389
499,15363,Apollo15,Crustal,Cataclastic,0.00050,71.00,0.000355
540,15415,Apollo15,Crustal,Anorthosite,0.26940,67.07,0.180687
557,15437,Apollo15,Crustal,Anorthosite,0.00100,80.00,0.000800
731,60057,Apollo16,Crustal,Anorthosite,0.00310,100.00,0.003100
733,60059,Apollo16,Crustal,Anorthosite,0.00105,100.00,0.001050
769,60619,Apollo16,Crustal,Anorthosite,0.02800,90.75,0.025410
802,61017,Apollo16,Crustal,Anorthosite,0.00262,100.00,0.002620
831,61226,Apollo16,Crustal,Anorthosite,0.00153,100.00,0.001530


### 今までの情報をまとめるneeded_samples_overviewデータフレームを作成し、石の種類のカラムを作成してください

In [34]:
needed_samples_overview=pd.DataFrame()

In [35]:
needed_samples_overview['Type']=needed_samples['Type'].unique()

In [36]:
needed_samples_overview

,Type
0,Basalt
1,Breccia
2,Crustal


### 次に種類ごとの重さを計算し、代入してください。その際、名前をtotal_weightに変更してください

In [37]:
needed_samples_overview=pd.merge(needed_samples_overview,needed_samples.groupby('Type')['Weight (kg)'].sum(),on='Type')
needed_samples_overview=needed_samples_overview.rename(columns={'Weight (kg)':'Total_weight(kg)'})
needed_samples_overview

,Type,Total_weight(kg)
0,Basalt,17.42340
1,Breccia,10.11850
2,Crustal,4.74469


### 宇宙飛行士がおおよその大きさを把握するために各種類ごとの平均重量を計算し、代入してください。その際、名前をAverage_weightに変更してください

In [38]:
needed_samples_overview=pd.merge(needed_samples_overview,needed_samples.groupby('Type')['Weight (kg)'].mean(),on='Type')
needed_samples_overview=needed_samples_overview.rename(columns={'Weight (kg)':'Average_weight(kg)'})
needed_samples_overview

,Type,Total_weight(kg),Average_weight(kg)
0,Basalt,17.42340,1.244529
1,Breccia,10.11850,1.264812
2,Crustal,4.74469,0.103145


### それぞれの石の種類の所有している数、それが全体に占める割合を計算したカラムを作成する。

In [39]:
samples=rock_samples[rock_samples['Type'].isin(['Basalt','Breccia','Crustal'])].groupby('Type')['ID'].count()
samples

Type
Basalt     351
Breccia    959
Crustal     46
Name: ID, dtype: int64

In [40]:
all_samples=rock_samples[rock_samples['Type'].isin(['Basalt','Breccia','Crustal'])].groupby('Type')['ID'].count().sum()
all_samples

1356

In [41]:
needed_samples.groupby('Type')['Weight (kg)'].count().sum()

68

In [42]:
samples/all_samples

Type
Basalt     0.258850
Breccia    0.707227
Crustal    0.033923
Name: ID, dtype: float64

In [43]:
needed_samples_overview=pd.merge(needed_samples_overview,samples,on='Type')
needed_samples_overview=needed_samples_overview.rename(columns={'ID':'サンプル数'})
needed_samples_overview

,Type,Total_weight(kg),Average_weight(kg),サンプル数
0,Basalt,17.42340,1.244529,351
1,Breccia,10.11850,1.264812,959
2,Crustal,4.74469,0.103145,46


In [44]:
needed_samples_overview=pd.merge(needed_samples_overview,samples/all_samples,on='Type')
needed_samples_overview=needed_samples_overview.rename(columns={'ID':'Percentage of rocks'})
needed_samples_overview

,Type,Total_weight(kg),Average_weight(kg),サンプル数,Percentage of rocks
0,Basalt,17.42340,1.244529,351,0.258850
1,Breccia,10.11850,1.264812,959,0.707227
2,Crustal,4.74469,0.103145,46,0.033923


### アルテミスミッションで持ち帰れると推定されるサンプル総量の平均をもとに各石の持ち帰れる重量を計算し、その重量を各石の平均重量で割ることで持ち帰れる各石の数量を計算してください

In [45]:
artemis_mission

,Mission,Total weight (kg),Payload (kg),Sample_weight_from_Total_weight,Sample_weight_from_payload,Estemated sample weight(kg)
0,artemis1,26520,26988,76.766734,38.779584,57.773159
1,artemis1b,26520,37965,76.766734,54.552649,65.659691
2,artemis2,26520,42955,76.766734,61.722877,69.244806


In [46]:
artemis_mission_avg=artemis_mission['Estemated sample weight(kg)'].mean()

In [47]:
needed_samples_overview['Weight to collect (kg)']=artemis_mission_avg*needed_samples_overview['Percentage of rocks']
needed_samples_overview

,Type,Total_weight(kg),Average_weight(kg),サンプル数,Percentage of rocks,Weight to collect (kg)
0,Basalt,17.42340,1.244529,351,0.258850,16.624842
1,Breccia,10.11850,1.264812,959,0.707227,45.422289
2,Crustal,4.74469,0.103145,46,0.033923,2.178754


In [48]:
needed_samples_overview['Rocks to collect']=needed_samples_overview['Weight to collect (kg)']/needed_samples_overview['Average_weight(kg)']

In [49]:
needed_samples_overview

,Type,Total_weight(kg),Average_weight(kg),サンプル数,Percentage of rocks,Weight to collect (kg),Rocks to collect
0,Basalt,17.42340,1.244529,351,0.258850,16.624842,13.358345
1,Breccia,10.11850,1.264812,959,0.707227,45.422289,35.912271
2,Crustal,4.74469,0.103145,46,0.033923,2.178754,21.123128
